## Implementing multinomial Naive Bayes classifier on ‘20 Newsgroups Dataset’

In [199]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\RAZORBLADE\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [200]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\RAZORBLADE\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [201]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import operator
import os
import nltk
from string import punctuation
import re
from nltk.corpus import stopwords

## Get the training data

In [202]:
mypath = r"C:\Users\RAZORBLADE\Desktop\20news-19997\20_newsgroups"
folders = [f for f in os.listdir(mypath)]
folders.sort()
folders

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Stop Words

Stop words are words that show up a lot in every document (e.g. prepositions and pronouns). Since stop words are of no use for us we will not consider them


In [203]:
nltk.download('stopwords')
nltk.download('punkt')
punctuations=list(punctuation)
stopWords=stopwords.words('english')
stopWords+=punctuations 
stopWords= set(stopWords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RAZORBLADE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\RAZORBLADE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Our Vocabulary/Feature Set

In [204]:
# vocab_dict will be a dictionary of the form {word: frequency} over all documents

data = {}
vocab_dict = {}
for folder in folders:
    data[folder] = []
    for doc in os.listdir(os.path.join(mypath, folder)):
        with open(os.path.join(mypath, folder,doc), 'r') as f:
            text = f.read()
            text = text.lower()
            # remove any word that has a non alphabetical character
            text = re.sub(r'[^a-z]', ' ', text)
            temp=text.split()
            data[folder].append(temp)
            for token in temp:
                if token in vocab_dict:
                    vocab_dict[token] += 1
                elif len(token) >=5 and token not in stopWords:
                    vocab_dict[token] = 1
                    
            

len(vocab_dict)


90926

In [205]:
len(data[folders[0]][1])

5430

## Final Feature List

In [206]:
# Sort the dictionary based on frequency of each 'possible' vocabulary word
sorted_vocab=sorted(vocab_dict.items(),key=operator.itemgetter(1),reverse=True)
# Choosing top 2000 vocab words as features
feature_list=[]
for key in sorted_vocab:
    feature_list.append(key[0])
feature_list=feature_list[0:200] # K = 200 (number of words in vocab)
feature_list = set(feature_list)
feature_list

{'access',
 'actually',
 'agate',
 'always',
 'american',
 'andrew',
 'another',
 'anyone',
 'anything',
 'apple',
 'around',
 'article',
 'atheism',
 'athos',
 'autos',
 'available',
 'baseball',
 'based',
 'believe',
 'berkeley',
 'better',
 'called',
 'cantaloupe',
 'center',
 'change',
 'children',
 'christian',
 'clipper',
 'colorado',
 'columbia',
 'computer',
 'control',
 'could',
 'course',
 'crabapple',
 'crypt',
 'culture',
 'darwin',
 'david',
 'different',
 'distribution',
 'drive',
 'either',
 'electronics',
 'email',
 'enough',
 'europa',
 'every',
 'evidence',
 'example',
 'files',
 'first',
 'following',
 'followup',
 'forsale',
 'found',
 'games',
 'gatech',
 'general',
 'geneva',
 'getting',
 'given',
 'going',
 'government',
 'graphics',
 'great',
 'group',
 'gtefsd',
 'hardware',
 'harvard',
 'heard',
 'history',
 'hockey',
 'however',
 'howland',
 'human',
 'image',
 'information',
 'internet',
 'israel',
 'jesus',
 'jewish',
 'keywords',
 'least',
 'lines',
 'litt

## Data Preparation

In [207]:
def prepare_data():
    '''
    return : a dataframe of columns as features and rows as documents
    '''
    df = pd.DataFrame(columns = feature_list)
    Y=[]
    for folder in folders:
        for doc in data[folder]:
            Y.append(folder)
            # Add a new row for every file
            df.loc[len(df)] = np.zeros(len(feature_list))
            for txt in doc:
                for word in txt.split():
                    if word in feature_list:
                        df.loc[len(df)-1,word] += 1
        
    # add Y as a column to dataframe
    df['Y']=Y
    return df

    

## Getting Test Data and Training Data

In [208]:
df = prepare_data()

In [209]:
X = df.drop(['Y'],axis=1).values
#remove last column of X

Y = df['Y'].values
df.head()


,maybe,rochester,jesus,looking,support,heard,wanted,network,apple,atheism,...,remember,source,getting,given,research,history,saying,electronics,culture,Y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,...,0.0,0.0,0.0,0.0,0.0,6.0,1.0,0.0,0.0,alt.atheism
1,1.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,57.0,...,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,alt.atheism
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,alt.atheism
3,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,alt.atheism
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,alt.atheism


In [223]:
# divide each row of df by sum of all values in that row except the last column
df[df.columns[:-1]] = df[df.columns[:-1]]/df[df.columns[:-1]].sum(axis=1)


ValueError: Columns must be same length as key

In [210]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=0,test_size=0.25)

In [211]:
# count each distinct element in the y_train
from collections import Counter
Counter(y_train)


Counter({'talk.politics.misc': 741,
         'talk.religion.misc': 764,
         'sci.electronics': 756,
         'misc.forsale': 739,
         'rec.motorcycles': 716,
         'comp.sys.mac.hardware': 764,
         'rec.sport.hockey': 769,
         'talk.politics.mideast': 719,
         'soc.religion.christian': 745,
         'sci.med': 744,
         'comp.os.ms-windows.misc': 751,
         'talk.politics.guns': 751,
         'alt.atheism': 767,
         'comp.graphics': 747,
         'comp.sys.ibm.pc.hardware': 760,
         'rec.sport.baseball': 752,
         'sci.crypt': 767,
         'rec.autos': 731,
         'sci.space': 754,
         'comp.windows.x': 760})

## Using the inbuilt Multinomial Naive Bayes

In [212]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(x_train,y_train)
clf.score(x_test,y_test)

0.767

## Posteriors

In [213]:
# calculate posteriors for each type of folder
posteriors ={}
for folder in folders:
    posteriors[folder] = len(y_train[y_train==folder]) / len(y_train)
    print(folder,posteriors[folder])


alt.atheism 0.05114356204574248
comp.graphics 0.04980996199239848
comp.os.ms-windows.misc 0.05007668200306728
comp.sys.ibm.pc.hardware 0.05067680202707208
comp.sys.mac.hardware 0.050943522037740885
comp.windows.x 0.05067680202707208
misc.forsale 0.04927652197106088
rec.autos 0.04874308194972328
rec.motorcycles 0.047742881909715276
rec.sport.baseball 0.05014336200573448
rec.sport.hockey 0.05127692205107688
sci.crypt 0.05114356204574248
sci.electronics 0.05041008201640328
sci.med 0.04960992198439688
sci.space 0.05027672201106888
soc.religion.christian 0.04967660198706408
talk.politics.guns 0.05007668200306728
talk.politics.mideast 0.047942921917716874
talk.politics.misc 0.04940988197639528
talk.religion.misc 0.050943522037740885


## CCD

In [214]:
word= 'example'
folder=folders[0]
(df[word][df['Y']==folder].sum() + 1) / (len(df[df['Y']==folder]) + len(feature_list))

0.22583333333333333

In [215]:
# calculate class conditional probabilities p(x|y) where x is a word and y is a newsgroup for each word in the vocabulary as a dataframe
conditional_probabilities = pd.DataFrame(columns=feature_list)
i=0;
for folder in folders:
    # add a new row for each folder
    conditional_probabilities.loc[len(conditional_probabilities)] = np.zeros(len(feature_list))
    # use df calculated above to calculate conditional probabilities
    for word in feature_list:
        #print(word)
        conditional_probabilities[word][i] = (df[word][df['Y']==folder].sum() + 1) / (len(df[df['Y']==folder]) + len(feature_list))
    i+=1
conditional_probabilities.head()


,maybe,rochester,jesus,looking,support,heard,wanted,network,apple,atheism,...,however,remember,source,getting,given,research,history,saying,electronics,culture
0,0.080833,0.017500,0.301667,0.039167,0.105000,0.056667,0.031667,0.040833,0.262500,1.520000,...,0.170833,0.060000,0.042500,0.050000,0.128333,0.037500,0.108333,0.184167,0.001667,0.036667
1,0.037500,0.175000,0.000833,0.187500,0.168333,0.060000,0.051667,0.104167,0.055000,0.000833,...,0.081667,0.023333,0.147500,0.045000,0.060833,0.173333,0.004167,0.027500,0.010833,0.005000
2,0.058333,0.194167,0.000833,0.092500,0.099167,0.048333,0.041667,0.134167,0.025833,0.001667,...,0.081667,0.051667,0.029167,0.052500,0.015833,0.060833,0.002500,0.017500,0.010000,0.005000
3,0.050833,0.174167,0.001667,0.075833,0.129167,0.068333,0.065833,0.098333,0.020000,0.000833,...,0.085000,0.028333,0.025833,0.078333,0.019167,0.059167,0.001667,0.011667,0.020833,0.000833
4,0.035833,0.200000,0.000833,0.065833,0.089167,0.086667,0.081667,0.116667,0.709167,0.000833,...,0.057500,0.033333,0.029167,0.077500,0.028333,0.042500,0.000833,0.020833,0.020833,0.000833


In [216]:
# normalize each columns of conditional_probabilities
conditional_probabilities = conditional_probabilities.div(conditional_probabilities.sum(axis=0), axis=1)
# get the sum of each column
conditional_probabilities.sum(axis=0)

maybe          1.0
rochester      1.0
jesus          1.0
looking        1.0
support        1.0
              ... 
research       1.0
history        1.0
saying         1.0
electronics    1.0
culture        1.0
Length: 200, dtype: float64

## Multinomial Naive Bayes Classifier

In [217]:
a={'a':1,'b':2,'c':3}
max(a,key=a.get)

'c'

In [218]:
def predict(text):
    # calculate the probability of each word in the text belonging to each newsgroup
    probabilities = {}
    i=0
    for f_in in range(len(folders)):
        probabilities[folders[f_in]] = np.log(posteriors[folders[f_in]])
        for i in range(len(text)):
            probabilities[folders[f_in]] += np.log(conditional_probabilities.values[f_in][i])*text[i]
    # return the newsgroup with the highest probability
    # print(probabilities)
    return max(probabilities, key=probabilities.get)

In [219]:
def predict_data(x_test):
    predictions = []
    for i in range(len(x_test)):
        print(':',end=" ")
        predictions.append(predict(x_test[i]))
    return predictions
    

In [220]:
from sklearn.metrics import classification_report,confusion_matrix
y_pred=predict_data(x_test)
# print(y_pred)

: : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : 

In [221]:
sum(y_pred==y_test)/len(y_test)

0.6192